In [1]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [2]:
import pandas as pd
from urlparse import urlparse
import tldextract
from collections import Counter
import jieba
import json
import math
import numpy as np
from time import time

In [3]:
df = pd.read_csv('/Users/nhu2000/projects/SimilarSearch/data/clean/huizhou-content-v1.csv')

In [4]:
df.info()

In [5]:
import re
match = re.compile('([a-zA-Z_0-9_=_#_&_/])') #remove letters, numbers, and special characters in the article body
#match = re.compile('([a-zA-Z_0-9])') #remove all of english letters and numbers
def classifier_filters(s):
    return match.sub(r' ', s) 

In [6]:
def filter_combined(x):
    s = x.split()
    r = [t for t in s if (t.strip()) > 0 and t.strip() not in keyword_stop_words]
    t = ' '.join(r)
    ret = classifier_filters(t)
    #print 'before -----', t
    #print 'after ------', ret
    return ret   

In [7]:
huizhou_stop_words = [u'的', u'了', u'在', u'是', u'安徽', u'和', u'有', u'我', u'年', u'上', u'他', u'我们', 
                      u'为', u'与', u'人', u'也', u'于', u'中', u'就', u'都', u'在线', u'等', u'不', u'网站', u'你', 
                      u'一个', u'到', u'之', u'对', u'说', u'进入', u'后', u'月', u'将', u'时', u'又', u'被', u'会员'
                     u'版权', u'论坛', u'以', u'而', u'着', u'下', u'这', u'但', u'要', u'或', u'首页', u'这',
                     u'地', u'从', u'卡', u'来', '多', u'日', u'她', u'还', u'联系', u'那', u'一处', u'一直', u'个', 
                     u'向', u'并', u'曾', u'这样', u'里', u'合肥', u'推荐', u'专栏', u'相关', u'当前', u'所有', u'中心'
                     u'频道', u'编辑', u'位置', u'第一', u'社区', u'看', '中安', u'安徽省', u'大', u'许可证', u'许可',
                     u'让', u'没有', u'什么', u'其', '请', '该', '用', '积分', u'凤阳', u'一篇', u'版权', u'县',
                     ]

s = ' '.join(huizhou_stop_words)
#s = u'444 （）'

In [15]:
import unicodedata
def remove_non_chinese(s):
    r = [c for c in s if unicodedata.category(c).startswith('L')]
#print r
    return ''.join(r)
#print remove_non_chinese(s)

hz_dict = dict(zip(words, fq))

In [16]:
def ch_title_letters_removal(x):
    #print 'coming', x
    t = x.strip('  ')
    if(len(t)) == 0:
        return None
    
    if(len(x) > 0):
        #s = x.strip()
        s = remove_non_chinese(x)
        if len(s) > 0 and s not in huizhou_stop_words:
            return s.strip()
        else:
            return None
    return None

In [17]:
class Toolbox (object):
    def __init__(self):
        self.init = 1
        
    def sort_dict (self, d, max=50):
        t = sorted(d.items(), key=lambda x: (-x[1], x[0]))
        return t[:max]
    
    def add_dict (self, x, y):
        d = { k: x.get(k, 0) + y.get(k, 0) for k in set(x) | set(y) }
        return d
    
    def create_dict(self, k1, v1, filters = True):
        if filters == True:
            k1_list0 = k1.strip('()').split(',')
            v1_list0 = v1.strip('()').split(',')
            #print v1_list0
            #print k1_list0
            k1_list1 = filter(None, k1_list0)        
            v1_list1 = filter(None, v1_list0)
        else:
            k1_list1 = k1
            v1_list1 = v1
        
        #print ' '.join(k1_list1)
        k2_list1 = [item.decode('utf-8') for item in k1_list1]
        v2_list1 = [float(item) for item in v1_list1]

    def create_vec(self, k1, v1):
        return (zip(k1, v1))
    
    def get_dict_top_n(self, d, max=50):
        ret = self.sort_dict(d, max)
        words, fq = zip(*ret)
        sum_up = sum(fq)
        fq_normalized = tuple(item/sum_up for item in fq)
        
        return dict(zip(words, fq_normalized))
    
    #calculate cosine between two dictionaries
    def get_sim(self, vec1, vec2):
        
        #get intersection
        intersection = set(vec1.keys()) & set(vec2.keys())
        
        #print intersection
        numerator = sum([vec1[x] * vec2[x] for x in intersection])

        #print 'num ---', numerator
        #print vec1.keys()
        #print vec1.values()
        #print vec2.keys()
        #print vec2.values()        
        
        sum1 = sum([vec1[x]**2 for x in vec1.keys()])
        sum2 = sum([vec2[x]**2 for x in vec2.keys()])
        denominator = math.sqrt(sum1) * math.sqrt(sum2)

        if not denominator:
            return 0.0
        else:
            return float(numerator) / denominator
        
    def sum_vects(self, vects):
        sum_vects = None
        for vec in vects:
            if sum_vects == None:
                sum_vects = vec
            else:
                sum_vects = self.add_dict(sum_vects, vec)
        #only return first 100 after additions
        return self.get_dict_top_n(sum_vects)
    
    def unserailzed_list(x, eval_val=False):
        #print '----- in'
        #print x
        s = x.strip('()')
        #s = s.strip('[]')
        ret = s.split(',')
        #print len(ret)
        #print 'split -----'
        #print ' '.join(ret)
        if eval_val == True:
            return tuple(float(item) for item in ret if len(item) > 0)        
        else:
            return tuple(item.strip().decode('utf-8') for item in ret if len(item) > 0)

In [18]:
global idx
idx = 0
def map_signature_vect(x): 
    global idx
    print idx
    idx += 1
    s = x.decode('utf-8').split()
    #print 'vect ---', len(s), s
    ret = [ch_title_letters_removal(item) for item in s if len(item) > 0 and ch_title_letters_removal(item) != None]
    #print 'ret ---', ret, len(ret)
    count = Counter(ret)
    
    #print 'count', len(count)
    
    top_50 = count.most_common(50)
    
    #print 'top_100', len(top_100)
    
    if len(top_50) > 0:
        #print 'going ---'
        words, cnt = zip(*top_50)
        return words, cnt
    else:
        #print 'warning'
        return None, None

In [19]:
def get_summary(x):
    s = x.split()
    ret = ' '.join(s[:50]) + '...'
    return ret

In [20]:
#calculte similarity
def calculate_sim(x, in_dict):
    #print in_dict
    tb = Toolbox()
    return tb.get_sim(x, in_dict)

In [21]:
#merge the dict
def map_merge_vec(k, v):
    return zip(k, v)

In [22]:
#merge the dict
def map_merge_dict(k, v):
    return dict(zip(k, v))

In [23]:
#create a signature
def map_signature(k, v):
    return zip(k, v)

In [24]:
#normalize the cnt
def map_normalize_cnt(x):
    tot = sum(x)
    return tuple(float(item)/tot for item in x)

In [17]:
new_df= df[:2000]
new_df.info()
df = new_df

In [21]:
check = df['content_seg'].map(map_signature_vect)    
df['s_words'], df['s_cnt'] = zip(*check)

In [260]:
df['s_vec'] = (map(map_merge_vec, df['s_words'],df['s_cnt']))

In [22]:
df['s_fq'] = df.s_cnt.map(lambda x: map_normalize_cnt(x))

In [353]:
df['s_dict'] = (map(map_merge_dict, df['s_words'],df['s_fq']))

In [23]:
df['s_signature'] = (map(map_signature, df['s_words'],df['s_cnt']))
df['s_sig'] = (map(map_signature, df['s_words'],df['s_fq']))

In [24]:
df['s_sig_serial'] = df.s_sig.map(lambda x: str(x) )

In [26]:
type(df.iloc[0]['s_sig_serial'])

str

In [406]:
new_df = df[['url', 'title','summary', 's_words', 's_cnt', 's_fq', 's_vec']]

In [407]:
new_df.to_csv('../data/svc/huizhou_signature_vec_v1.csv', encoding='utf-8', index=False)

In [266]:
new_df.head()

,url,title,summary,s_words,s_cnt,s_fq
0,http://cul.anhuinews.com/system/2016/11/28/007...,洪建华夫妇的竹刻人生,谁能想到，洪建华夫妇这对一无学历、二无家传、三无师承的“三无大师”成了省级非物质...,"(竹刻, 建华, 雕刻, 洪, 文化, 作品, 非, 传承, 工艺, 收藏, 徽派, 他们,...","(31, 22, 21, 20, 17, 16, 12, 12, 11, 11, 10, 1...","[0.0756097560976, 0.0536585365854, 0.051219512..."
1,http://cul.anhuinews.com/system/2016/11/21/007...,徽州民歌传人操明花校园授艺,近日，省非物质文化遗产徽州民歌传承人操明花前往休宁县海阳二小、歙县城关小学和徽州...,"(民歌, 徽州, 们, 磨麦, 文化, 会员, 中国, 磨米, 传承, 同学, 徽文化, 方...","(8, 6, 5, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, ...","[0.0909090909091, 0.0681818181818, 0.056818181..."
2,http://cul.anhuinews.com/system/2016/11/03/007...,黄山楹联漫谈,黄山楹联第一人当推明朝诗人余绍祉。他遍历祖国名山大川，见到无数奇峰怪石，却感到抵...,"(黄山, 笔者, 文联, 才, 怪石, 旅游, 一联, 编写, 画, 奇峰怪石, 赖少, 后...","(10, 4, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,...","[0.119047619048, 0.047619047619, 0.03571428571..."
3,http://cul.anhuinews.com/system/2016/01/26/007...,徽派雕刻——徽州木雕艺术,徽州民居、家具木板和圆木雕刻艺术的简称。木雕在旧属徽州各县分布之广在...,"(木雕, 徽州, 艺术, 家具, 吉祥图案, 少数, 显示, 内, 木板, 雕刻, 宅第, ...","(6, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.101694915254, 0.0508474576271, 0.0338983050..."
4,http://cul.anhuinews.com/system/2016/01/26/007...,徽派雕刻——徽州砖雕艺术,徽州砖雕徽州砖雕的造型，来源于汉画象砖。汉画象砖大都是模印砖坯刻画后入...,"(砖雕, 艺术, 题材, 风格, 狮子, 民间, 砖, 图案, 画象, 美术, 为主, 徽州...","(10, 5, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2,...","[0.0961538461538, 0.0480769230769, 0.038461538..."


In [126]:
new_df.info()

In [275]:
df_slim = pd.read_csv('../data/svc/huizhou_signature_vec_v1.csv')

In [276]:
df_slim.info()

In [277]:
df_slim_no_dup = df_slim.drop_duplicates(['s_vec'])

In [278]:
df_slim_no_dup.info()

In [279]:
df_slim_no_dup.to_csv('../data/svc/huizhou_signature_vec_no_dup_v1.csv', encoding='utf-8', index=False)

In [4]:
df_open = pd.read_csv('../data/svc/huizhou_signature_vec_no_dup_v1.csv')

from ast import literal_eval
df_open['s_cnt'] = df_open.s_cnt.map(lambda x: eval(x))

In [5]:
def eval_utf8 (x, eval_val = False):
    #print '----- in'
    #print x
    s = x.strip('()')
    s = s.strip('[]')
    ret = s.split(',')
    #print len(ret)
    #print 'split -----'
    #print ' '.join(ret)
    if eval_val == True:
        return tuple(float(item) for item in ret if len(item) > 0)        
    else:
        return tuple(item.strip().decode('utf-8') for item in ret if len(item) > 0)

In [238]:
print type(eval_utf8(df_open.iloc[1]['s_cnt']))

In [6]:
df_open['s_words'] = df_open.s_words.map(lambda x: eval_utf8(x))

In [7]:
df_open['s_cnt'] = df_open.s_cnt.map(lambda x: eval_utf8(x), True)

In [8]:
df_open['s_fq'] = df_open.s_fq.map(lambda x: eval_utf8(x, True))

In [16]:
df_open['s_dict'] = (map(map_merge_dict, df_open['s_words'],df_open['s_fq']))

In [359]:
df_open.head()

,url,title,summary,s_words,s_cnt,s_fq,s_vec,s_dict,s_sim
0,http://cul.anhuinews.com/system/2016/11/28/007...,洪建华夫妇的竹刻人生,谁能想到，洪建华夫妇这对一无学历、二无家传、三无师承的“三无大师”成了省级非物质...,"(竹刻, 建华, 雕刻, 洪, 文化, 作品, 非, 传承, 工艺, 收藏...","(31, 22, 21, 20, 17, 16, 12, 12, 11, ...","(0.0756097560976, 0.0536585365854, 0.051219512...","[(竹刻, 31), (建华, 22), (雕刻, 21), (洪, 20), (文化, 1...","{u' 金奖': 0.00975609756098, u' 藏家': 0.007317073...",0.0
1,http://cul.anhuinews.com/system/2016/11/21/007...,徽州民歌传人操明花校园授艺,近日，省非物质文化遗产徽州民歌传承人操明花前往休宁县海阳二小、歙县城关小学和徽州...,"(民歌, 徽州, 们, 磨麦, 文化, 会员, 中国, 磨米, 传承, 同...","(8, 6, 5, 3, 3, 3, 3, 3, 3, 2, 2, 2...","(0.0909090909091, 0.0681818181818, 0.056818181...","[(民歌, 8), (徽州, 6), (们, 5), (磨麦, 3), (文化, 3), (...","{u' 进校园': 0.0113636363636, u' 越': 0.0113636363...",0.0
2,http://cul.anhuinews.com/system/2016/11/03/007...,黄山楹联漫谈,黄山楹联第一人当推明朝诗人余绍祉。他遍历祖国名山大川，见到无数奇峰怪石，却感到抵...,"(黄山, 笔者, 文联, 才, 怪石, 旅游, 一联, 编写, 画, 奇峰...","(10, 4, 3, 3, 2, 2, 2, 2, 2, 2, 2, ...","(0.119047619048, 0.047619047619, 0.03571428571...","[(黄山, 10), (笔者, 4), (文联, 3), (才, 3), (怪石, 2), ...","{u' 赖少': 0.0238095238095, u' 下联': 0.0119047619...",0.0
3,http://cul.anhuinews.com/system/2016/01/26/007...,徽派雕刻——徽州木雕艺术,徽州民居、家具木板和圆木雕刻艺术的简称。木雕在旧属徽州各县分布之广在...,"(木雕, 徽州, 艺术, 家具, 吉祥图案, 少数, 显示, 内, 木板, ...","(6, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1...","(0.101694915254, 0.0508474576271, 0.0338983050...","[(木雕, 6), (徽州, 3), (艺术, 2), (家具, 2), (吉祥图案, 1)...","{u' 梓': 0.0169491525424, u' 则': 0.016949152542...",0.0
4,http://cul.anhuinews.com/system/2016/01/26/007...,徽派雕刻——徽州砖雕艺术,徽州砖雕徽州砖雕的造型，来源于汉画象砖。汉画象砖大都是模印砖坯刻画后入...,"(砖雕, 艺术, 题材, 风格, 狮子, 民间, 砖, 图案, 画象, 美...","(10, 5, 4, 4, 3, 3, 3, 3, 3, 2, 2, ...","(0.0961538461538, 0.0480769230769, 0.038461538...","[(砖雕, 10), (艺术, 5), (题材, 4), (风格, 4), (狮子, 3),...","{u' 厅': 0.00961538461538, u' 严谨': 0.0096153846...",0.0


In [27]:
df_open.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190283 entries, 0 to 190282
Data columns (total 9 columns):
url        190283 non-null object
title      190283 non-null object
summary    190283 non-null object
s_words    190283 non-null object
s_cnt      190283 non-null object
s_fq       190283 non-null object
s_vec      190283 non-null object
s_dict     190283 non-null object
s_sim      190283 non-null float64
dtypes: float64(1), object(8)
memory usage: 13.1+ MB


In [17]:
#Get contents for '徽州'
test_dict = dict()
test_dict[u'徽州'] = 0.5
test_dict[u'徽文化'] = 0.5
t_sim = 's_sim'

In [18]:
d = df_open.iloc[3]['s_dict']
print ' '.join(d.keys())

吉祥图案 少数 显示 内 木板 雕刻 杉树 选材 楠木 床 用具 情调 梓 分布 使用 则 为贵 题材 宅院 戏曲 栏柱 民间 图案 以柏 故事 榧 之广在 旧属 徽州 木雕 由 各县 民居 艺术家 圆木 吉祥 著名 椿 艺术 人物 均 全国 案 参与 花鸟 家具 宗教 宅第 椅 术家


In [19]:
calculate_sim(df_open.iloc[3]['s_dict'], test_dict)

NameError: global name 'Toolbox' is not defined

In [299]:
#let us find the cetroid for given data set of 'huizhou'

In [22]:
t0 = time()
df_open[t_sim] = df_open.s_dict.map(lambda x: calculate_sim(x, test_dict))
check = df_open[df_open[t_sim] > 0.01]
print 'after s_sim', time() - t0

if len(check) > 2000:
    #sampling 
    ratio = 2000*1.0/len(check)
    msk = np.random.rand(len(check)) < ratio
    tmp = check[msk]
    check = tmp

check.info()

check_vecs = check['s_dict']
tb = Toolbox()
centroid_vec = tb.sum_vects(check_vecs) 
print 'after centroid', time() - t0
#only work on non duplicated ones
df_open[t_sim] = df_open.s_dict.map(lambda x: calculate_sim(x, centroid_vec))
s_sim = df_open[t_sim].values.argsort()[::-1]
print time() - t0

after s_sim 2.7563829422
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1983 entries, 5 to 190165
Data columns (total 9 columns):
url        1983 non-null object
title      1983 non-null object
summary    1983 non-null object
s_words    1983 non-null object
s_cnt      1983 non-null object
s_fq       1983 non-null object
s_vec      1983 non-null object
s_dict     1983 non-null object
s_sim      1983 non-null float64
dtypes: float64(1), object(8)
memory usage: 154.9+ KB
after centroid 13.5589950085
18.4071319103


In [23]:
df_open['s_sim'].max()

0.85913767352080439

In [302]:
df_open.head()

,url,title,summary,s_words,s_cnt,s_fq,s_vec,s_dict,s_sim
0,http://cul.anhuinews.com/system/2016/11/28/007...,洪建华夫妇的竹刻人生,谁能想到，洪建华夫妇这对一无学历、二无家传、三无师承的“三无大师”成了省级非物质...,"(竹刻, 建华, 雕刻, 洪, 文化, 作品, 非, 传承, 工艺, 收藏...","(31, 22, 21, 20, 17, 16, 12, 12, 11, 11, 10, 1...","[0.0756097560976, 0.0536585365854, 0.051219512...","[(竹刻, 31), (建华, 22), (雕刻, 21), (洪, 20), (文化, 1...","{u' 金奖': 4, u' 藏家': 3, u' 因为': 4, u' 十多': 5, u...",0.0
1,http://cul.anhuinews.com/system/2016/11/21/007...,徽州民歌传人操明花校园授艺,近日，省非物质文化遗产徽州民歌传承人操明花前往休宁县海阳二小、歙县城关小学和徽州...,"(民歌, 徽州, 们, 磨麦, 文化, 会员, 中国, 磨米, 传承, 同...","(8, 6, 5, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, ...","[0.0909090909091, 0.0681818181818, 0.056818181...","[(民歌, 8), (徽州, 6), (们, 5), (磨麦, 3), (文化, 3), (...","{u' 进校园': 1, u' 越': 1, u' 原汁': 1, u' 上口': 1, u...",0.0
2,http://cul.anhuinews.com/system/2016/11/03/007...,黄山楹联漫谈,黄山楹联第一人当推明朝诗人余绍祉。他遍历祖国名山大川，见到无数奇峰怪石，却感到抵...,"(黄山, 笔者, 文联, 才, 怪石, 旅游, 一联, 编写, 画, 奇峰...","(10, 4, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,...","[0.119047619048, 0.047619047619, 0.03571428571...","[(黄山, 10), (笔者, 4), (文联, 3), (才, 3), (怪石, 2), ...","{u' 赖少': 2, u' 下联': 1, u' 见到': 2, u' 其下': 2, u...",0.0
3,http://cul.anhuinews.com/system/2016/01/26/007...,徽派雕刻——徽州木雕艺术,徽州民居、家具木板和圆木雕刻艺术的简称。木雕在旧属徽州各县分布之广在...,"(木雕, 徽州, 艺术, 家具, 吉祥图案, 少数, 显示, 内, 木板, ...","(6, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.101694915254, 0.0508474576271, 0.0338983050...","[(木雕, 6), (徽州, 3), (艺术, 2), (家具, 2), (吉祥图案, 1)...","{u' 梓': 1, u' 则': 1, u' 图案': 1, u' 情调': 1, u' ...",0.0
4,http://cul.anhuinews.com/system/2016/01/26/007...,徽派雕刻——徽州砖雕艺术,徽州砖雕徽州砖雕的造型，来源于汉画象砖。汉画象砖大都是模印砖坯刻画后入...,"(砖雕, 艺术, 题材, 风格, 狮子, 民间, 砖, 图案, 画象, 美...","(10, 5, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2,...","[0.0961538461538, 0.0480769230769, 0.038461538...","[(砖雕, 10), (艺术, 5), (题材, 4), (风格, 4), (狮子, 3),...","{u' 厅': 1, u' 严谨': 1, u' 狮子': 3, u' 内容': 2, u'...",0.0


In [397]:
dsi_centroid = centroid_vec
#save centroid
import json
with open('../data/centroids/dsi_centroid.json', 'w') as fp:
    json.dump(dsi_centroid, fp)

In [24]:
#print out centroid 
ret = tb.sort_dict(centroid_vec)
words, cnt = zip(*ret)
for i in range(0, len(words)):
    print i, words[i], cnt[i]

0 文化 0.140679077091
1 徽州 0.128655915232
2 婺源 0.0393812507091
3 中国 0.037581546178
4 名人 0.0341781989495
5 徽文化 0.0332302878834
6 历史 0.0293176299789
7 照片 0.0277366346702
8 研究 0.0257577358725
9 老照片 0.0246818780733
10 黄山 0.0227673735535
11 版权 0.0215068797318
12 徽商 0.0211181170094
13 休宁 0.0208601704393
14 状元 0.019451783451
15 旅游 0.0182395254341
16 作家 0.0174310538887
17 建筑 0.0174018711144
18 频道 0.0164269421371
19 详细 0.0156404014333
20 臺 0.0137204247014
21 黄梅 0.0131095640318
22 专题 0.0128213430832
23 毕业 0.0121858331822
24 手艺 0.0117159231714
25 信息 0.0112732802438
26 艺术 0.0112204911481
27 新闻 0.0111970898641
28 中心 0.0110257479898
29 发展 0.0110176887115
30 人民 0.0106120798718
31 歙县 0.0104890846324
32 徽派 0.0103187171123
33 女人 0.0102532261543
34 戏曲 0.009102929669
35 新安 0.00893903219872
36 安庆 0.00883776911621
37 文学 0.00877192497846
38 保护 0.00856499825144
39 作者 0.00826604621256
40 屯溪 0.00805151451874
41 奥运 0.00791533203958
42 徽学 0.00780151180449
43 原创 0.00736573521306
44 回复 0.00731832692569
45 吃 0.0073144

In [412]:
ratio = 0.50
df_out = df_open[df_open[t_sim] > ratio]
df_out.info()

In [25]:
nlp = df_open[df_open[t_sim] == 0.0]

In [26]:
nlp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36063 entries, 9 to 190103
Data columns (total 9 columns):
url        36063 non-null object
title      36063 non-null object
summary    36063 non-null object
s_words    36063 non-null object
s_cnt      36063 non-null object
s_fq       36063 non-null object
s_vec      36063 non-null object
s_dict     36063 non-null object
s_sim      36063 non-null float64
dtypes: float64(1), object(8)
memory usage: 2.8+ MB


In [418]:
s_sim = df_out['s_sim'].values.argsort()[::-1]

In [419]:
idx = 0
for i in s_sim:
    if df_out.iloc[i]['s_sim'] > ratio:
        title = df_out.iloc[i]['title']
        sim_val = df_out.iloc[i][t_sim]
        idx += 1
        print idx, i, title, sim_val

In [420]:
def get_centroid_df(df, t_sim, ratio):
    check = df[df[t_sim] > ratio]

    check.info()
    check_vecs = check['s_dict']
    tb = Toolbox()
    return tb.sum_vects(check_vecs) 

In [433]:
centroid_vec = get_centroid_df(df_out, t_sim, 0.50)

In [430]:
print sum(ret_df.values())

In [ ]:
check = df_open[df_open[t_sim] > 0.01]
print 'after s_sim', time() - t0

if len(check) > 2000:
    #sampling 
    ratio = 2000*1.0/len(check)
    msk = np.random.rand(len(check)) < ratio
    tmp = check[msk]
    check = tmp

check.info()

check_vecs = check['s_dict']
tb = Toolbox()
centroid_vec = tb.sum_vects(check_vecs) 
print 'after centroid', time() - t0
#only work on non duplicated ones
df_open[t_sim] = df_open.s_dict.map(lambda x: calculate_sim(x, centroid_vec))
s_sim = df_open[t_sim].values.argsort()[::-1]
print time() - t0